## Example script to generate .json configuration file required for GA optimization.

In [1]:
import os
import pandas as pd

### GA hyperparameters

In [2]:
n_organisms = 128 #number of organisms
n_elites = 8 #number of elite organisms
n_generations = 2 #number of generations

config = {'n_organisms': n_organisms,
          'n_elites': n_elites,
          'n_generations': n_generations,
          
          'crossover_rate': 1.0,
          'mutation_rate': 1.0,
          'gamma': 0.05,
         }

In [3]:
output_folder_name = 'OUTPUT'
output_folder_path = '../../results/'

config['output_folder_name'] = os.path.join(output_folder_path, output_folder_name)

### Fitness function

Fitness function is declared in ga/mpi_scripts/loss_utils.py . Currently, either RMSE of the signal or the weighted superposition of the RMSE of the signal and its derivative is supported (RMSE, RMSE_GRAD correspondingly). 

In [4]:
config['loss'] = 'RMSE'

### Input data

Folders containg CSV data files of voltage-clamp protocol and recorded patch-clamp traces are indicated below. Protocol folder is supposed to contain .csv file with the protocol. Initial state protocol folder should contain .csv file where the holding potential between sweeps is saved after import from .abf file (002_Patch_clamp_output_files_preprocessing_for_ga.ipynb). Weight for trace and/or weight for trace derivative should be saved in weight_dir. By default all weights are equal to 1.

In [5]:
trace_dir = '../data/traces/'# patch-clamp recorded trace
protocol_dir = '../data/protocols/' # voltage clamp protocol
initial_state_protocol_dir = '../data/initial_state_protocols/' # initial state protocol

weight_dir = '../data/weights/' # weight for trace

In [6]:
### Traces
traces = pd.DataFrame(os.listdir(trace_dir), columns=['Trace filename'])
traces

,Trace filename
0,NEW_NAME.csv
1,inactivation.csv
2,activation.csv


In [7]:
### Protocols

protocols = pd.DataFrame(os.listdir(protocol_dir), columns=['Protocol filename'])
protocols

,Protocol filename
0,NEW_NAME.csv
1,inactivation.csv
2,activation.csv


In [8]:
### Initial state protocols

initial_state_protocols = pd.DataFrame(os.listdir(initial_state_protocol_dir), columns=['initial state protocol filename'])
initial_state_protocols

,initial state protocol filename
0,NEW_NAME.csv
1,activation_initial_state.csv
2,inactivation_initial_state.csv


In [9]:
### Weights

weights = pd.DataFrame(os.listdir(weight_dir), columns=['Weight filename'])
weights

,Weight filename
0,activation.csv


In [10]:
# Input data directories contents is shown above.

trace_filename = 'activation.csv' # filename of trace
protocol_filename = 'activation.csv' # filename of protocol
initial_state_protocol_filename = 'activation_initial_state.csv' # filename of initial state protocol
weight_filename = 'activation.csv' # filename of weight

trace_path = os.path.abspath(os.path.join(trace_dir, trace_filename))
protocol_path = os.path.abspath(os.path.join(protocol_dir, protocol_filename))
initial_state_protocol_path = os.path.abspath(os.path.join(initial_state_protocol_dir, 
                                                           initial_state_protocol_filename))
weight_path = os.path.abspath(os.path.join(weight_dir, weight_filename))



individual = {}
individual['filename_phenotype'] = trace_path
individual['filename_protocol'] = protocol_path
individual['filename_initial_state_protocol'] = initial_state_protocol_path
individual['filename_sample_weight'] = weight_path

If initial protocol wasn't generated from .abf file, instead of ['filename_initial_state_protocol'] parameter, it should be specified by name ['initial_state_protocol']. ['initial_state_protocol'] parameter should contain holding potential ("v") and time between sweeps in the protocol("t"). Example is shown below.

In [21]:
# initial_state_v = -80.0# mV
# initial_state_t = 1.# s
# individual['initial_state_protocol'] = {'v':initial_state_v,
#                                        't':initial_state_t}

## Model
The model of experimental setup should be compiled as a 'filename_so' C library. Model state variables, algebraic variables and constants are described in 'filename_legend_states', 'filename_legend_algebraic' and 'filename_legend_constants' .csv files correspondingly. The lists of variables and constants is stored as a SUNDIALS NVector internally, with the order corresponding to the respective csv files. The name of output current state variable is controlled by 'columns_model' parameter of the .json configuraration file.

In [12]:
dirname_model = '../src/model_ctypes/PC_model/'
filename_so = 'libina.so'

config['filename_so'] =  os.path.abspath(os.path.join(dirname_model, filename_so))
config['filename_legend_states'] =  os.path.abspath(os.path.join(dirname_model, 'legend_states.csv'))
config['filename_legend_algebraic'] =  os.path.abspath(os.path.join(dirname_model, 'legend_algebraic.csv'))
config['filename_legend_constants'] =  os.path.abspath(os.path.join(dirname_model, 'legend_constants.csv'))

config['columns_model'] = 'I_out'

In [13]:
config

{'n_organisms': 128,
 'n_elites': 8,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/OUTPUT',
 'loss': 'RMSE',
 'filename_so': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/libina.so',
 'filename_legend_states': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_algebraic': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv',
 'filename_legend_constants': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'columns_model': 'I_out'}

## Free parameters and their boundaries

In [14]:
config['experimental_conditions'] = {}

Free parameters are divided in two parts: experimental setup parameters designated as 'individual' parameters, and ionic current parameters designated as 'individual' parameters. Both lists are expected to be not empty. Absoulte scale for the parameter and its boundaries is used if 'is_multiplier' boolean is set to False. Otherwise parameter value is considered to be a scaler of original model value, logarithmic scale is used in this case. 

NOTE: Simultaneous optimization of multiple experimental traces is supported. In this case 'common' parameters are supposed to be the same in every experiment, while 'individual' are supposed to depend on particular recording. Because of that the recorded trace and the protocol filenames are stored as 'individual' parameters as well. Change 'individual' to 'individual#' in this case, where # corresponds to the number of the trace.

In [15]:
individual['params'] = {
                   'c_m': {'bounds': [0.1, 5.0],  'is_multiplier': True},
                   'R': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'g_max': {'bounds': [0.05, 100.0], 'is_multiplier': True},
                   'g_leak': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'x_c_comp': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'x_r_comp': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'alpha': {'bounds': [0.72, 0.78], 'is_multiplier': False},
                   'v_rev': {'bounds': [16.0, 80.0], 'is_multiplier': False},
                   'tau_z': {'bounds': [0.4, 6.0], 'is_multiplier': True}}

In [16]:
default_settings = {'bounds': [0.1, 10], 'is_multiplier': True}

common = {}
common_parameters = ['a0_m','b0_m','delta_m','s_m', 'tau_m_const',
                    'a0_h', 'b0_h','delta_h','s_h', 'tau_h_const',
                    'a0_j', 'b0_j', 'delta_j', 's_j', 'tau_j_const',
                    ]


# Using default settings
common['params'] = {param: default_settings for param in common_parameters}

# You can change bounds for free parameters if it's necessary
common['params']['tau_m_const']['bounds'] =  [0.1, 2.0]





In [17]:
common['params']['v_half_h'] = {'bounds': [60.0, 80.0], 'is_multiplier': False}
common['params']['v_half_m'] = {'bounds': [20.0, 30.0], 'is_multiplier': False}
common['params']['k_h'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}
common['params']['k_m'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}


In [18]:
config['experimental_conditions']['common'] = common
config['experimental_conditions']['individual'] = individual

In [19]:
config

{'n_organisms': 128,
 'n_elites': 8,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/OUTPUT',
 'loss': 'RMSE',
 'filename_so': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/libina.so',
 'filename_legend_states': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_algebraic': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv',
 'filename_legend_constants': '/home/nik/Documents/WORK/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'columns_model': 'I_out',
 'experimental_conditions': {'common': {'params': {'a0_m': {'bounds': [0.1,
      2.0],
     'is_multiplier': True},
    'b0_m': {'bounds': [0.1, 2.0], 'is_multiplier': True},
    'delta_m': {'bounds': [0.1, 2.0], 'is_multiplier': True},
    's_m': {'bounds': [0.1, 2.0], 'is_multiplier': True},
    'tau_m_const': {'bounds': [0.1, 2.0], 'is_multiplier': True},
    '

## Save config

In [20]:
import json

config_name = 'NEW_NAME'
with open(f"../ga/configs/{config_name}.json", 'w') as f:
    f.write(json.dumps(config, indent=4))